<a href="https://colab.research.google.com/github/Ezhil2318/Pandas_Practice/blob/main/pandasPractice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
employees_df = pd.DataFrame({
    'emp_id': [101, 102, 103, 104, 105, 106],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank'],
    'dept_id': [1, 2, 1, 3, 2, 3],
    'salary': [55000, 62000, 52000, 58000, 60000, 54000],
    'join_date': pd.to_datetime(['2021-01-10', '2020-06-23', '2021-03-12', '2020-09-01', '2022-07-15', '2022-03-03']),
    'manager_id': [None, 101, 101, 102, 102, 104]
})

departments_df = pd.DataFrame({
    'dept_id': [1, 2, 3, 4],
    'dept_name': ['HR', 'Finance', 'IT', 'Marketing'],
    'location': ['Mumbai', 'Delhi', 'Chennai', 'Bangalore']
})

projects_df = pd.DataFrame({
    'proj_id': [301, 302, 303, 304, 305, 306],
    'proj_name': ['Onboarding App', 'Budget Tracker', 'Compliance Portal', 'Website Revamp', 'Forecast Tool', 'Audit Automation'],
    'emp_id': [101, 102, 103, 104, 105, 106],
    'dept_id': [1, 2, 1, 3, 2, 3],
    'hours_worked': [120, 95, 110, 150, 85, 130],
    'deadline': pd.to_datetime(['2023-12-01', '2023-11-15', '2023-10-30', '2024-01-15', '2023-09-05', '2023-12-20'])
})

ADVANCED LEVEL QUESTIONS - 1

ANTI JOIN

In [ ]:
x = employees_df.merge(departments_df,on='dept_id',how='inner')
x = x[['emp_id','name','dept_id','dept_name','location']]
xPro = x.merge(projects_df,on=['emp_id','dept_id'],how='inner')
xPro[['name','dept_name','proj_name','location']]

,name,dept_name,proj_name,location
0,Alice,HR,Onboarding App,Mumbai
1,Bob,Finance,Budget Tracker,Delhi
2,Charlie,HR,Compliance Portal,Mumbai
3,David,IT,Website Revamp,Chennai
4,Eva,Finance,Forecast Tool,Delhi
5,Frank,IT,Audit Automation,Chennai


In [ ]:
selfJoin = employees_df.merge(employees_df,left_on='manager_id',right_on='emp_id',how='left',suffixes=('','_mgr'))
selfJoin[['emp_id','name','manager_id','name_mgr']]

,emp_id,name,manager_id,name_mgr
0,101,Alice,NaN,NaN
1,102,Bob,101.0,Alice
2,103,Charlie,101.0,Alice
3,104,David,102.0,Bob
4,105,Eva,102.0,Bob
5,106,Frank,104.0,David


In [ ]:
depPro = departments_df.merge(projects_df,on='dept_id',how='outer',indicator=True)
depPro[depPro['_merge']=='left_only'][['dept_name','location']]

,dept_name,location
6,Marketing,Bangalore


In [ ]:
employees_df.iloc[1::2]

,emp_id,name,dept_id,salary,join_date,manager_id
1,102,Bob,2,62000,2020-06-23,101.0
3,104,David,3,58000,2020-09-01,102.0
5,106,Frank,3,54000,2022-03-03,104.0


In [ ]:
employees_df[(employees_df['salary']>employees_df['salary'].quantile(0.9))&(employees_df['join_date'].dt.year>2021)]

,emp_id,name,dept_id,salary,join_date,manager_id


In [ ]:
x = employees_df.select_dtypes(include='number').columns
i = [employees_df.columns.get_loc(col) for col in x]
employees_df.iloc[:,i]

,emp_id,dept_id,salary,manager_id
0,101,1,55000,NaN
1,102,2,62000,101.0
2,103,1,52000,101.0
3,104,3,58000,102.0
4,105,2,60000,102.0
5,106,3,54000,104.0


In [ ]:
projects_df[projects_df.deadline.between('2023-10-01','2023-12-31')]

,proj_id,proj_name,emp_id,dept_id,hours_worked,deadline
0,301,Onboarding App,101,1,120,2023-12-01
1,302,Budget Tracker,102,2,95,2023-11-15
2,303,Compliance Portal,103,1,110,2023-10-30
5,306,Audit Automation,106,3,130,2023-12-20


In [ ]:
employees_df.groupby('dept_id').agg({'salary':('count','mean'),'join_date':'min'}).reset_index()

dept_id salary           join_date
           count     mean        min
0       1      2  53500.0 2021-01-10
1       2      2  61000.0 2020-06-23
2       3      2  56000.0 2020-09-01

In [ ]:
employees_df.assign(dept_avg_salary = employees_df.groupby('dept_id')['salary'].transform('mean'))

,emp_id,name,dept_id,salary,join_date,manager_id,dept_avg_salary
0,101,Alice,1,55000,2021-01-10,NaN,53500.0
1,102,Bob,2,62000,2020-06-23,101.0,61000.0
2,103,Charlie,1,52000,2021-03-12,101.0,53500.0
3,104,David,3,58000,2020-09-01,102.0,56000.0
4,105,Eva,2,60000,2022-07-15,102.0,61000.0
5,106,Frank,3,54000,2022-03-03,104.0,56000.0


In [ ]:
depPro = departments_df.merge(projects_df,on='dept_id',how='inner')
x = depPro.groupby(['dept_id','dept_name'])['hours_worked'].sum().reset_index(name="total_hrs_worked")
x['ranked'] = x['total_hrs_worked'].rank(ascending=False)
x

,dept_id,dept_name,total_hrs_worked,ranked
0,1,HR,230,2.0
1,2,Finance,180,3.0
2,3,IT,280,1.0


In [ ]:
x = projects_df[projects_df['hours_worked']>100]
print("Total no.of projects with 100+ work hour=",x.shape[0])

Total no.of projects with 100+ work hour= 4


In [ ]:
employees_df.sort_values(['dept_id','salary'],ascending=(True,False))

,emp_id,name,dept_id,salary,join_date,manager_id
0,101,Alice,1,55000,2021-01-10,NaN
2,103,Charlie,1,52000,2021-03-12,101.0
1,102,Bob,2,62000,2020-06-23,101.0
4,105,Eva,2,60000,2022-07-15,102.0
3,104,David,3,58000,2020-09-01,102.0
5,106,Frank,3,54000,2022-03-03,104.0


In [ ]:
empPro = employees_df.merge(projects_df,on='emp_id',how='inner')
empPro['ranked'] = empPro['hours_worked'].rank(ascending=False)
empPro[['emp_id','name','ranked']].sort_values('ranked')

,emp_id,name,ranked
3,104,David,1.0
5,106,Frank,2.0
0,101,Alice,3.0
2,103,Charlie,4.0
1,102,Bob,5.0
4,105,Eva,6.0


ADVANCED LEVEL QUESTION - 2

In [ ]:
empEmp = employees_df.merge(employees_df,left_on='manager_id',right_on='emp_id',how='inner',suffixes=("","_mgr"))
x = empEmp.groupby(['manager_id','name_mgr']).agg({'salary':'sum','emp_id':'count'}).rename(columns={'salary':'tot_sal_mged','emp_id':'tot_emp_count'}).reset_index()
x.sort_values('tot_emp_count',ascending=False)

,manager_id,name_mgr,tot_sal_mged,tot_emp_count
0,101.0,Alice,114000,2
1,102.0,Bob,118000,2
2,104.0,David,54000,1


In [ ]:
employees_df[employees_df.join_date.dt.quarter==1]

,emp_id,name,dept_id,salary,join_date,manager_id
0,101,Alice,1,55000,2021-01-10,NaN
2,103,Charlie,1,52000,2021-03-12,101.0
5,106,Frank,3,54000,2022-03-03,104.0


In [ ]:
empEmp[empEmp['join_date']<empEmp['join_date_mgr']].iloc[:,0:6]

,emp_id,name,dept_id,salary,join_date,manager_id
0,102,Bob,2,62000,2020-06-23,101.0


In [ ]:
x = employees_df.merge(projects_df,on='emp_id',how='inner')
x = x.select_dtypes(include='number')
del x['salary']
x[x['hours_worked']>100].head(3)

,emp_id,dept_id_x,manager_id,proj_id,dept_id_y,hours_worked
0,101,1,NaN,301,1,120
2,103,1,101.0,303,1,110
3,104,3,102.0,304,3,150


In [ ]:
x = employees_df.merge(departments_df,on='dept_id',how='inner')
joinAll = x.merge(projects_df,on='emp_id',how='inner')
joinAll['dept_avg_salary'] = joinAll.groupby('dept_id_x')['salary'].transform('mean')
pd.concat((joinAll[((joinAll.dept_name=='Finance')&(joinAll.salary>joinAll.dept_avg_salary))|(joinAll.hours_worked>100)].iloc[:,:6],joinAll.dept_avg_salary),axis=1)

,emp_id,name,dept_id_x,salary,join_date,manager_id,dept_avg_salary
0,101.0,Alice,1.0,55000.0,2021-01-10,NaN,53500.0
1,102.0,Bob,2.0,62000.0,2020-06-23,101.0,61000.0
2,103.0,Charlie,1.0,52000.0,2021-03-12,101.0,53500.0
3,104.0,David,3.0,58000.0,2020-09-01,102.0,56000.0
5,106.0,Frank,3.0,54000.0,2022-03-03,104.0,56000.0
4,NaN,NaN,NaN,NaN,NaT,NaN,61000.0


In [ ]:
empPro = employees_df.merge(projects_df,on='emp_id',how='inner',suffixes=('','_proj'))
empPro.sort_values('hours_worked', ascending=False).groupby('dept_id').head(1)

,emp_id,name,dept_id,salary,join_date,manager_id,proj_id,proj_name,dept_id_proj,hours_worked,deadline
3,104,David,3,58000,2020-09-01,102.0,304,Website Revamp,3,150,2024-01-15
0,101,Alice,1,55000,2021-01-10,NaN,301,Onboarding App,1,120,2023-12-01
1,102,Bob,2,62000,2020-06-23,101.0,302,Budget Tracker,2,95,2023-11-15


In [ ]:
x = pd.to_datetime(employees_df['join_date'].agg('min'))
employees_df[employees_df.join_date==x]

,emp_id,name,dept_id,salary,join_date,manager_id
1,102,Bob,2,62000,2020-06-23,101.0


In [ ]:
empCopy = employees_df.copy()
x = empCopy.sort_values('salary')
x.assign(sal_rank = x.groupby('dept_id')['salary'].rank())

,emp_id,name,dept_id,salary,join_date,manager_id,sal_rank
2,103,Charlie,1,52000,2021-03-12,101.0,1.0
5,106,Frank,3,54000,2022-03-03,104.0,1.0
0,101,Alice,1,55000,2021-01-10,NaN,2.0
3,104,David,3,58000,2020-09-01,102.0,2.0
4,105,Eva,2,60000,2022-07-15,102.0,1.0
1,102,Bob,2,62000,2020-06-23,101.0,2.0


In [ ]:
employees_df.groupby('dept_id').head()

,emp_id,name,dept_id,salary,join_date,manager_id
0,101,Alice,1,55000,2021-01-10,NaN
1,102,Bob,2,62000,2020-06-23,101.0
2,103,Charlie,1,52000,2021-03-12,101.0
3,104,David,3,58000,2020-09-01,102.0
4,105,Eva,2,60000,2022-07-15,102.0
5,106,Frank,3,54000,2022-03-03,104.0
